## Create _C.virginia_ long, non-coding RNA files.

### Downloads files from NCBI.

### Notebook relies on:

- [GffRead](https://github.com/gpertea/gffread)

- [GFFutils](https://gffutils.readthedocs.io/en/v0.12.0/index.html) available in your `$PATH`.

  - I accomplished this by creating/activating a conda environment for [GFFutils](https://gffutils.readthedocs.io/en/v0.12.0/index.html) and running this notebook from within that environment.

- [samtools](http://www.htslib.org/).

### Resulting files will be used for [_C.virginica_ RNAseq/DML sex/OA project](https://github.com/epigeneticstoocean/2018_L18-adult-methylation) (GitHub repo)

### List computer specs

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Fri 18 Feb 2022 07:10:15 AM PST
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.3 LTS
Release:	20.04
Codename:	focal

------------
HOSTNAME: 
computer

------------
Computer Specs:

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   45 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              1
Core(s) per socket:              1
Socket(s):                       2
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           165
Model name:                      Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz
Stepping:                        2
CPU MHz:                         2400.008
BogoMIPS:                        4800.01
Hypervisor vendor:               VMware
Virtualization type:    

No LSB modules are available.


### Set variables
- `%env` indicates a bash variable

- without `%env` is Python variable

In [2]:
# Set directories, input/output files
%env data_dir=/home/sam/data/C_virginica/genomes
%env analysis_dir=/home/sam/analyses/20220217-cvir-lncRNA_subsetting
analysis_dir="20220217-cvir-lncRNA_subsetting"

# Input files (from NCBI)
%env ncbi_fasta=GCF_002022765.2_C_virginica-3.0_genomic.fna
%env ncbi_fasta_index=GCF_002022765.2_C_virginica-3.0_genomic.fna.fai
%env ncbi_fasta_gz=GCF_002022765.2_C_virginica-3.0_genomic.fna.gz
%env ncbi_gff=GCF_002022765.2_C_virginica-3.0_genomic.gff
%env ncbi_gff_gz=GCF_002022765.2_C_virginica-3.0_genomic.gff.gz

# URL to download files from NCBI
%env ncbi_url=https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/022/765/GCF_002022765.2_C_virginica-3.0

# Output files
%env lncRNA_bed=GCF_002022765.2_C_virginica-3.0_lncRNA.bed
%env lncRNA_gff=GCF_002022765.2_C_virginica-3.0_lncRNA.gff
%env lncRNA_gtf=GCF_002022765.2_C_virginica-3.0_lncRNA.gtf
%env lncRNA_fasta=GCF_002022765.2_C_virginica-3.0_lncRNA.fa
%env lncRNA_fasta_index=GCF_002022765.2_C_virginica-3.0_lncRNA.fa.fai

# Set program locations
%env gffread=/home/sam/programs/gffread-0.12.7.Linux_x86_64/gffread
%env samtools=/home/sam/programs/samtools-1.12/samtools

env: data_dir=/home/sam/data/C_virginica/genomes
env: analysis_dir=/home/sam/analyses/20220217-cvir-lncRNA_subsetting
env: ncbi_fasta=GCF_002022765.2_C_virginica-3.0_genomic.fna
env: ncbi_fasta_index=GCF_002022765.2_C_virginica-3.0_genomic.fna.fai
env: ncbi_fasta_gz=GCF_002022765.2_C_virginica-3.0_genomic.fna.gz
env: ncbi_gff=GCF_002022765.2_C_virginica-3.0_genomic.gff
env: ncbi_gff_gz=GCF_002022765.2_C_virginica-3.0_genomic.gff.gz
env: ncbi_url=https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/022/765/GCF_002022765.2_C_virginica-3.0
env: lncRNA_bed=GCF_002022765.2_C_virginica-3.0_lncRNA.bed
env: lncRNA_gff=GCF_002022765.2_C_virginica-3.0_lncRNA.gff
env: lncRNA_gtf=GCF_002022765.2_C_virginica-3.0_lncRNA.gtf
env: lncRNA_fasta=GCF_002022765.2_C_virginica-3.0_lncRNA.fa
env: lncRNA_fasta_index=GCF_002022765.2_C_virginica-3.0_lncRNA.fa.fai
env: gffread=/home/sam/programs/gffread-0.12.7.Linux_x86_64/gffread
env: samtools=/home/sam/programs/samtools-1.12/samtools


### Create analysis directory

In [3]:
%%bash
# Make analysis directory, if it doesn't exist
mkdir --parents "${analysis_dir}"

### Download GFF

In [4]:
%%bash
cd "${data_dir}"

# Download with wget.
# Use --quiet option to prevent wget output from printing too many lines to notebook
# Use --continue to prevent re-downloading fie if it's already been downloaded.
wget --quiet \
--continue \
${ncbi_url}/${ncbi_gff_gz}

# Unzip download GFF
gunzip "${ncbi_gff_gz}"

ls -ltrh "${ncbi_gff}"

-rw-rw-r-- 1 sam sam 412M Dec 10  2019 GCF_002022765.2_C_virginica-3.0_genomic.gff


gzip: GCF_002022765.2_C_virginica-3.0_genomic.gff already exists;	not overwritten


### Examine GFF

In [5]:
%%bash
head -n 20 "${data_dir}"/"${ncbi_gff}"

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build C_virginica-3.0
#!genome-build-accession NCBI_Assembly:GCF_002022765.2
#!annotation-source NCBI Crassostrea virginica Annotation Release 100
##sequence-region NC_035780.1 1 65668440
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=6565
NC_035780.1	RefSeq	region	1	65668440	.	+	.	ID=NC_035780.1:1..65668440;Dbxref=taxon:6565;Name=1;chromosome=1;collection-date=22-Mar-2015;country=USA;gbkey=Src;genome=chromosome;isolate=RU13XGHG1-28;isolation-source=Rutgers Haskin Shellfish Research Laboratory inbred lines (NJ);mol_type=genomic DNA;tissue-type=whole sample
NC_035780.1	Gnomon	gene	13578	14594	.	+	.	ID=gene-LOC111116054;Dbxref=GeneID:111116054;Name=LOC111116054;gbkey=Gene;gene=LOC111116054;gene_biotype=lncRNA
NC_035780.1	Gnomon	lnc_RNA	13578	14594	.	+	.	ID=rna-XR_002636969.1;Parent=gene-LOC111116054;Dbxref=GeneID:111116054,Genbank:XR_002636969.1;Name=XR_002636969.1;gbkey=ncRNA;gene=LOC111

### Download NCBI genomic FastA

In [6]:
%%bash
cd "${data_dir}"

# Download with wget.
# Use --quiet option to prevent wget output from printing too many lines to notebook
# Use --continue to prevent re-downloading fie if it's already been downloaded.
wget --quiet \
--continue \
${ncbi_url}/${ncbi_fasta_gz}

# Unzip download GFF
gunzip "${ncbi_fasta_gz}"

ls -ltrh "${ncbi_fasta}"


-rw-rw-r-- 1 sam sam 662M Dec 10  2019 GCF_002022765.2_C_virginica-3.0_genomic.fna


gzip: GCF_002022765.2_C_virginica-3.0_genomic.fna already exists;	not overwritten


### Create FastA index with [Samtools](http://www.htslib.org/)

In [7]:
%%bash
cd "${data_dir}"

${samtools} faidx "${ncbi_fasta}"

ls -ltrh "${ncbi_fasta_index}"

-rw-rw-r-- 1 sam sam 398 Feb 18 07:10 GCF_002022765.2_C_virginica-3.0_genomic.fna.fai


### Inspect NCBI genomic FastA index

In [8]:
%%bash
cd "${data_dir}"

head "${ncbi_fasta_index}"

NC_035780.1	65668440	117	80	81
NC_035781.1	61752955	66489530	80	81
NC_035782.1	77061148	129014514	80	81
NC_035783.1	59691872	207039044	80	81
NC_035784.1	98698416	267477182	80	81
NC_035785.1	51258098	367409446	80	81
NC_035786.1	57830854	419308388	80	81
NC_035787.1	75944018	477862245	80	81
NC_035788.1	104168038	554755681	80	81
NC_035789.1	32650045	660225938	80	81


### Extracts lncRNAs from genomic GFF using `gtf_extract` from [GFFutils](https://gffutils.readthedocs.io/en/v0.12.0/index.html)

In [9]:
%%bash
cd "${data_dir}"

# Capture GFF header from NCBI gff
head -n 7 "${ncbi_gff}" > ${analysis_dir}/"${lncRNA_gff}"

# Add note about modification
printf "#%s%s\n" "!" "lncRNA only - created by Sam White $(date)" >> ${analysis_dir}/"${lncRNA_gff}"


# Finds lncRNAs in NCBI GFF
gtf_extract \
--feature lnc_RNA \
--gff "${ncbi_gff}" \
>> ${analysis_dir}/"${lncRNA_gff}"


head ${analysis_dir}/"${lncRNA_gff}"

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build C_virginica-3.0
#!genome-build-accession NCBI_Assembly:GCF_002022765.2
#!annotation-source NCBI Crassostrea virginica Annotation Release 100
##sequence-region NC_035780.1 1 65668440
#!lncRNA only - created by Sam White Fri 18 Feb 2022 07:10:32 AM PST
NC_035780.1	Gnomon	lnc_RNA	13578	14594	.	+	.	ID=rna-XR_002636969.1;Parent=gene-LOC111116054;Dbxref=GeneID:111116054,Genbank:XR_002636969.1;Name=XR_002636969.1;gbkey=ncRNA;gene=LOC111116054;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 1 sample with support for all annotated introns;product=uncharacterized LOC111116054;transcript_id=XR_002636969.1
NC_035780.1	Gnomon	lnc_RNA	169468	170178	.	-	.	ID=rna-XR_002635081.1;Parent=gene-LOC111105702;Dbxref=GeneID:111105702,Genbank:XR_002635081.1;Name=XR_002635081.1;gbkey=ncRNA;gene=LOC111105702;model_evidence=Supporting evi

### Extract lncRNAs to BED using [GffRead](https://github.com/gpertea/gffread)

In [10]:
%%bash
cd "${data_dir}"

${gffread} --bed \
${analysis_dir}/"${lncRNA_gff}" \
> ${analysis_dir}/"${lncRNA_bed}"

### Inspect lncRNA BED

In [11]:
%%bash
head ${analysis_dir}/"${lncRNA_bed}"

NC_035780.1	13577	14594	rna-XR_002636969.1	100	+	13577	14594	0,0,0	1	1017,	0,	geneID=gene-LOC111116054;gene_name=LOC111116054
NC_035780.1	169467	170178	rna-XR_002635081.1	100	-	169467	170178	0,0,0	1	711,	0,	geneID=gene-LOC111105702;gene_name=LOC111105702
NC_035780.1	900325	903430	rna-XR_002636046.1	100	+	900325	903430	0,0,0	1	3105,	0,	geneID=gene-LOC111111519;gene_name=LOC111111519
NC_035780.1	1280830	1282416	rna-XR_002638148.1	100	-	1280830	1282416	0,0,0	1	1586,	0,	geneID=gene-LOC111124195;gene_name=LOC111124195
NC_035780.1	1432943	1458091	rna-XR_002639675.1	100	+	1432943	1458091	0,0,0	1	25148,	0,	geneID=gene-LOC111135942;gene_name=LOC111135942
NC_035780.1	1503801	1513830	rna-XR_002636574.1	100	-	1503801	1513830	0,0,0	1	10029,	0,	geneID=gene-LOC111114441;gene_name=LOC111114441
NC_035780.1	1856840	1863683	rna-XR_002636864.1	100	-	1856840	1863683	0,0,0	1	6843,	0,	geneID=gene-LOC111115591;gene_name=LOC111115591
NC_035780.1	1856840	1863697	rna-XR_002636863.1	100	-	1856840	1863697	0,0,0	1	

### Convert lncRNA GFF to GTF

In [12]:
%%bash
cd "${data_dir}"

${gffread} -E \
${analysis_dir}/"${lncRNA_gff}" -T \
1> ${analysis_dir}/"${lncRNA_gtf}" \
2> ${analysis_dir}/gffread-lncRNA_gff-to-lncRNA_gtf.stderr

### Inspect lncRNA GTF

In [13]:
%%bash
head ${analysis_dir}/"${lncRNA_gtf}"

NC_035780.1	Gnomon	transcript	13578	14594	.	+	.	transcript_id "rna-XR_002636969.1"; gene_id "gene-LOC111116054"; gene_name "LOC111116054"
NC_035780.1	Gnomon	exon	13578	14594	.	+	.	transcript_id "rna-XR_002636969.1"; gene_id "gene-LOC111116054"; gene_name "LOC111116054";
NC_035780.1	Gnomon	transcript	169468	170178	.	-	.	transcript_id "rna-XR_002635081.1"; gene_id "gene-LOC111105702"; gene_name "LOC111105702"
NC_035780.1	Gnomon	exon	169468	170178	.	-	.	transcript_id "rna-XR_002635081.1"; gene_id "gene-LOC111105702"; gene_name "LOC111105702";
NC_035780.1	Gnomon	transcript	900326	903430	.	+	.	transcript_id "rna-XR_002636046.1"; gene_id "gene-LOC111111519"; gene_name "LOC111111519"
NC_035780.1	Gnomon	exon	900326	903430	.	+	.	transcript_id "rna-XR_002636046.1"; gene_id "gene-LOC111111519"; gene_name "LOC111111519";
NC_035780.1	Gnomon	transcript	1280831	1282416	.	-	.	transcript_id "rna-XR_002638148.1"; gene_id "gene-LOC111124195"; gene_name "LOC111124195"
NC_035780.1	Gnomon	exon	1280831	12824

### Exract lncRNAs to FastA

Explanation of GffRead options used below:

- `-w`: specifies output FastA file

- `-W`: specifies to write coordinates of all exons spliced in FastA deflines

- `-g`: specifies input FastA (needs to have a corresponding FastA index file in same directory)

In [14]:
%%bash
cd "${data_dir}"

${gffread} -E \
-w ${analysis_dir}/"${lncRNA_fasta}" -W \
-g "${ncbi_fasta}" \
${analysis_dir}/"${lncRNA_gtf}" \
2> ${analysis_dir}/gffread_lncRNA-fasta-extraction.stderr

### Inspect lncRNA FastA

In [15]:
%%bash
head ${analysis_dir}/"${lncRNA_fasta}"

>rna-XR_002636969.1 loc:NC_035780.1|13578-14594|+ exons:13578-14594 segs:1-1017
tgatattgttgtgtGCAGAACGTggggtaagaaaacatgcaacactcataatattttacaatctgtctaG
TTTTCGTTGGACACATCCCACATACTAGAGGAAGGTCAGAAGCATGGGGGTGGTGGCATgctttttacac
tgaatgatcggcagtttgcagtgttcaactccaaatctcttctatgcacaaatcaaataacaaactttac
aCAGCTGTTACATGGAAAGTacctacatattttcataatggaaagaaataattatgaccatcacactgta
ttgaatttactagagaatatattgacttagaaggtttttttttaactttgtactggctgccaggcatgat
aacatgctacatcatacatgttgacttttaatcatcttaatagaagtaaaaacaataaaggtaatctctc
tgaaataaacttttattgatgaatgcattgatatgtatacatgtatgtcatcacagttttctcactatca
ttcctgaaatgtacagtgtcagctgatgtcatgatgatctacattttacataaaaattttcctCCTGAGA
TAAAAAGCGCAGATTAATATTTCACTCAATCccattttaactgttttattatacatattaactcttaaac


### Create lncRNA FastA index

In [16]:
%%bash
cd "${analysis_dir}"

${samtools} faidx "${lncRNA_fasta}"

ls -ltrh "${lncRNA_fasta_index}"

-rw-rw-r-- 1 sam sam 179K Feb 18 07:11 GCF_002022765.2_C_virginica-3.0_lncRNA.fa.fai


### Inspect lncRNA FastA index

In [17]:
%%bash
cd "${analysis_dir}"

head "${lncRNA_fasta_index}"

rna-XR_002636969.1	1017	80	70	71
rna-XR_002635081.1	711	1195	70	71
rna-XR_002636046.1	3105	2001	70	71
rna-XR_002638148.1	1586	5239	70	71
rna-XR_002639675.1	25148	6937	70	71
rna-XR_002636574.1	10029	32534	70	71
rna-XR_002636864.1	6843	42795	70	71
rna-XR_002636863.1	6857	49824	70	71
rna-XR_002635698.1	5581	56867	70	71
rna-XR_002637875.1	1611	62616	70	71


### Generate checksums

In [18]:
%%bash
cd "${analysis_dir}"

for file in *
do
  md5sum "${file}" | tee --append checksums.md5
done

28de37c9ee1308ac1175397d16b3aafe  GCF_002022765.2_C_virginica-3.0_lncRNA.bed
7fac9e7191915f763cc7f5d22838ac25  GCF_002022765.2_C_virginica-3.0_lncRNA.fa
1b43db284950abc07afb5f50164fb264  GCF_002022765.2_C_virginica-3.0_lncRNA.fa.fai
00755b8c80166cdec94b09f231ef440a  GCF_002022765.2_C_virginica-3.0_lncRNA.gff
dedab056acd679cf4eab83629882ee10  GCF_002022765.2_C_virginica-3.0_lncRNA.gtf
7ec412a022f43cfeb7729e55aac78ef6  gffread_lncRNA-fasta-extraction.stderr
cba3ae8e2474861cd60aa304269b66a8  gffread-lncRNA_gff-to-lncRNA_gtf.stderr


### Document GffRead program options

In [19]:
%%bash
${gffread} -h

gffread v0.12.7. Usage:
gffread [-g <genomic_seqs_fasta> | <dir>] [-s <seq_info.fsize>] 
 [-o <outfile>] [-t <trackname>] [-r [<strand>]<chr>:<start>-<end> [-R]]
 [--jmatch <chr>:<start>-<end>] [--no-pseudo] 
 [-CTVNJMKQAFPGUBHZWTOLE] [-w <exons.fa>] [-x <cds.fa>] [-y <tr_cds.fa>]
 [-j ][--ids <IDs.lst> | --nids <IDs.lst>] [--attrs <attr-list>] [-i <maxintron>]
 [--stream] [--bed | --gtf | --tlf] [--table <attrlist>] [--sort-by <ref.lst>]
 [<input_gff>] 

 Filter, convert or cluster GFF/GTF/BED records, extract the sequence of
 transcripts (exon or CDS) and more.
 By default (i.e. without -O) only transcripts are processed, discarding any
 other non-transcript features. Default output is a simplified GFF3 with only
 the basic attributes.
 
Options:
 --ids discard records/transcripts if their IDs are not listed in <IDs.lst>
 --nids discard records/transcripts if their IDs are listed in <IDs.lst>
 -i   discard transcripts having an intron larger than <maxintron>
 -l   discard transcripts

CalledProcessError: Command 'b'${gffread} -h\n'' returned non-zero exit status 1.

### Document `gtf_extract` options

In [20]:
%%bash
gtf_extract -h

usage: gtf_extract [-h] [-v] [-f FEATURE_TYPE] [--fields FIELD_LIST]
                   [-o OUTFILE] [--gff] [-k]
                   GTF_FILE

Extract selected data items from a GTF file and output in tab-delimited
format. The program can also operate on GFF files provided the --gff option is
specified.

positional arguments:
  GTF_FILE              input GTF file to extract data items from

optional arguments:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit
  -f FEATURE_TYPE, --feature FEATURE_TYPE
                        only extract data for lines where feature is
                        FEATURE_TYPE
  --fields FIELD_LIST   comma-separated list of fields to output in tab-
                        delimited format for each line in the GTF, e.g.
                        'chrom,start,end'. Fields can either be a GTF field
                        name (i.e. 'chrom', 'source', 'feature', 'start',
                       